## Load BOM

In [1]:
import os
import pandas as pd
import numpy as np
import requests
import time as t
import json
from tqdm import tqdm
from copy import deepcopy
import warnings
warnings.filterwarnings('ignore')

# from dotenv import load_dotenv
# load_dotenv()

# List Type: Values, Output Format: CSV
# Make sure no semi-colons are present in part descriptions
BOM_path = f"../AK4493-Combi10.csv"
# BOM_path = f"../AK4493-v1.64_26.csv"

BOM_df = pd.read_csv(BOM_path, delimiter = ";")
search_string = 'Unnamed'
columns_containing_string = [col for col in BOM_df.columns if search_string in col]
BOM_df.drop(columns = columns_containing_string, inplace = True)
display(BOM_df)
print("Done")

,Qty,Value,Device,Package,Parts,Description,ARROW_PART_NUMBER,ARROW_PRICE-STOCK,DESCRIPTION,HEIGHT,MANUFACTURER,MANUFACTURER_NAME,MANUFACTURER_PART_NUMBER,MOUSER_PART_NUMBER,MOUSER_PRICE-STOCK,POPULARITY,PROD_ID,SPICEPREFIX
0,4,NaN,"BZX79-C16,113",DIOAD1165W56L425D185,"Z1, Z2, Z5, Z6","Nexperia Zener Diode, 16V +/-2 %, +/-5 % 500 m...",NaN,NaN,"Nexperia Zener Diode, 16V +/-2 %, +/-5 % 500 m...",mm,NaN,Nexperia,"BZX79-C16,113",NaN,NaN,NaN,NaN,NaN
1,2,NaN,CONN_06SILK_FEMALE_PTH,1X06,"J2, J13",Multi connection point. Often used as Generic ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CONN-08437,NaN
2,6,0.1u,EMF212B7104KGHT,CAPC2012X135N,"C21A, C21B, C21B2, C21B4, C21B6, C21D",Multilayer Ceramic Capacitors MLCC - SMD/SMT 0...,NaN,NaN,Multilayer Ceramic Capacitors MLCC - SMD/SMT 0...,1.35mm,NaN,TAIYO YUDEN,EMF212B7104KGHT,963-EMF212B7104KGHT,https://www.mouser.co.uk/ProductDetail/Taiyo-Y...,NaN,NaN,NaN
3,2,"0.1u, Ceramic Capacitor X7R",EMF212B7104KGHT,CAPC2012X135N,"C28A, C28A1",Multilayer Ceramic Capacitors MLCC - SMD/SMT 0...,NaN,NaN,Multilayer Ceramic Capacitors MLCC - SMD/SMT 0...,1.35mm,NaN,TAIYO YUDEN,EMF212B7104KGHT,963-EMF212B7104KGHT,https://www.mouser.co.uk/ProductDetail/Taiyo-Y...,NaN,NaN,NaN
4,2,"0.1uF, 35V",MKS4B031002A00JC00,MKS4B031002A00JC00,"C2O, C2O1",Film Capacitors MKS 4 0.1 uF 50 VDC 2.5x7x10 P...,NaN,NaN,Film Capacitors MKS 4 0.1 uF 50 VDC 2.5x7x10 P...,7mm,NaN,WIMA,MKS4B031002A00JC00,505-MKS4B0312AJC,https://www.mouser.co.uk/ProductDetail/WIMA/MK...,NaN,NaN,NaN
5,2,100k,R-US_0411/15,0411/15,"R5P, R5P1","RESISTOR, American symbol",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,R
6,8,100uF,UFW1H101MPD1TD,CAPPRD350W60D825H1300,"C11O, C12O, C13O, C14O, C15O, C24O, C25O, C26O",Aluminum Electrolytic Capacitors - Radial Lead...,UFW1H101MPD1TD,https://www.arrow.com/en/products/ufw1h101mpd1...,Aluminum Electrolytic Capacitors - Radial Lead...,13mm,NaN,Nichicon,UFW1H101MPD1TD,647-UFW1H101MPD1TD,https://www.mouser.co.uk/ProductDetail/Nichico...,NaN,NaN,NaN
7,10,10k,LR1F10K,0207/10,"R11O, R19O, R20O, R20O1, R20O2, R21O, R22O, R2...","Metal film resistor, 10K 0.6W TE Connectivity ...",NaN,NaN,"Metal film resistor, 10K 0.6W TE Connectivity ...",mm,NaN,TE Connectivity,LR1F10K,279-LR1F10K,https://www.mouser.co.uk/ProductDetail/TE-Conn...,NaN,NaN,NaN
8,6,10u,EMK212BB7106KG-T,CAPC2012X135N,"C20A, C20B, C20B2, C20B4, C20B6, C20D",Multilayer Ceramic Capacitors MLCC - SMD/SMT 1...,NaN,NaN,Multilayer Ceramic Capacitors MLCC - SMD/SMT 1...,1.35mm,NaN,TAIYO YUDEN,EMK212BB7106KG-T,963-EMK212BB7106KG-T,https://www.mouser.co.uk/ProductDetail/TAIYO-Y...,NaN,NaN,NaN
9,4,15k,CPF215K000FKE14,0411/15,"R5, R11, R19, R20",VISHAY - CPF215K000FKE14 - Through Hole Resist...,NaN,NaN,VISHAY - CPF215K000FKE14 - Through Hole Resist...,mm,NaN,Vishay,CPF215K000FKE14,71-CPF2-F-15K-E3,https://www.mouser.com/ProductDetail/71-CPF2-F...,NaN,NaN,NaN


Done


## Define search_part function

In [8]:
def search_part(part_name):
    MOUSER_API_KEY = "c7b47461-c777-4dbc-8413-9bb69528b78f"
    
    headers = {
        'accept': 'application/json',
        'Content-Type': 'application/json',
    }
    
    params = {
        'apiKey': MOUSER_API_KEY,
    }
    
    json_data = {
        'SearchByPartRequest': {
            'mouserPartNumber': part_name,
            'partSearchOptions': 'string',
            'mouserPaysCustomsAndDuties': False,
        },
    }

    try_again_flag = True
    max_attempts = 10
    attempts = 0
    
    while try_again_flag and attempts < max_attempts:
        response = requests.post('https://api.mouser.com/api/v1/search/partnumber', params=params, headers=headers, json=json_data)
    
        t.sleep(6)
    
        match response.status_code:
            case 200:
                data = response.json()
                try_again_flag = False
            case 404:
                print("Resource not found (404 Not Found).")
                attempts += 1
            case 403:
                print("Permission denied (403 Forbidden).")
                attempts += 1
            case response.status_code if 300 <= response.status_code < 400:
                print(f"Redirection occurred (Status: {response.status_code}).")
                attempts += 1
            case response.status_code if 500 <= response.status_code < 600:
                print(f"Server error occurred (Status: {response.status_code}).")
                attempts += 1
            case _:
                print(f"Unexpected status code: {response.status_code} - {response.text}")
                attempts += 1

    if attempts == max_attempts:
        print(f"Max attempts reached - {attempts} / {max_attempts}.")
        return
    
    df = pd.DataFrame()
    
    # Sort and append new data
    for part in data["SearchResults"]["Parts"]:
        df = pd.concat([df, pd.json_normalize(part)], ignore_index=True)

    columns_to_keep = [
        # 'Availability',
        'DataSheetUrl',
        'Description',
        'Category',
        'LeadTime',
        'LifecycleStatus',
        'Manufacturer',
        'ManufacturerPartNumber',
        'Min',
        'Mult',
        'MouserPartNumber',
        'PriceBreaks',
        'SuggestedReplacement',
        'AvailabilityInStock',
    ]

    for column in df.columns:
        if not column in columns_to_keep:
            df.drop([column], axis = 1, inplace = True)
        else:
            pass

    if df.empty:
        return 30 # 30 == part not found
    elif len(df.columns) != len(columns_to_keep):
        missing_columns = list(set(columns_to_keep) - set(df.columns))
        if len(missing_columns) > 1:
            print(missing_columns)
            return 0 # 0 == missing multiple columns

        match missing_columns[0]:
            case 'Availability':
                return 1
            case 'DataSheetUrl':
                return 2
            case 'Description':
                return 3
            case 'Category':
                return 4
            case 'LeadTime':
                return 5
            case 'LifecycleStatus':
                return 6
            case 'Manufacturer':
                return 7
            case 'ManufacturerPartNumber':
                return 8
            case 'Min':
                return 9
            case 'Mult':
                return 10
            case 'MouserPartNumber':
                return 11
            case 'PriceBreaks':
                return 12
            case 'SuggestedReplacement':
                return 13
            case 'AvailabilityInStock':
                return 14
            case _:
                return 15

    if ("PriceBreaks" in df.columns) and not (df["PriceBreaks"].apply(lambda x: len(x) == 0).all()):
        df_exploded = df.explode("PriceBreaks").reset_index(drop=True)
        pricing_df = pd.json_normalize(df_exploded["PriceBreaks"])
        df_combined = pd.concat([df_exploded.drop(columns=["PriceBreaks"]), pricing_df], axis=1)
        df = df_combined.set_index(["MouserPartNumber", "Quantity"]).sort_index()
    else:
        return 20 # 20 == "PriceBreaks" not in column list or is invalid length
    
    for column in df.columns:
        try:
            df[column] = pd.to_numeric(df[column])
        except Exception as e:
            pass
    
    df = df[df["AvailabilityInStock"] > 0]
    if len(df.index.get_level_values(0)) == 0:
        return 31 # 31 == part not in stock
    
    df = df[df["Min"] == 1]
    if len(df.index.get_level_values(0)) == 0:
        return 32 # 32 == must buy more than 1
        
    df = df[df["Mult"] == 1]
    df = df.sort_values(by=['Quantity', 'Price'])
    
    if len(df.index.get_level_values(0)) > 0:
        return df.loc[[df.index.get_level_values(0)[0]]]
    else:
        return 33 # 33 == must buy in multiples > 1

print("Done")

Done


## Search Parts

In [9]:
results = pd.DataFrame()
pared_BOM = pd.DataFrame()
missing_parts = pd.DataFrame()

with tqdm(total = len(BOM_df["MOUSER_PART_NUMBER"].values), unit = "part") as pbar:
    for idx, row in BOM_df.iterrows():
        if row["MOUSER_PART_NUMBER"] is not np.nan:
            part_result = search_part(row["MOUSER_PART_NUMBER"])
        else:
            part_result = search_part(row["Device"])

        if isinstance(part_result, pd.DataFrame):
            results = pd.concat([results, pd.concat([part_result.reset_index(), pd.concat([pd.DataFrame(row).T] * len(part_result), ignore_index=True)], axis = 1)], axis = 0)
        elif isinstance(part_result, int):
            new_row = row
            new_row["error_code"] = part_result
            missing_parts = pd.concat([missing_parts, new_row], axis = 1)
        else:
            print("Invalid return from search_part function")
        pbar.update(1)

results = results.set_index(["MouserPartNumber", "Quantity"]).sort_index()
display(results)
print(f"Cant find {len(missing_parts.T)} part(s):")
display(missing_parts.T)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 42/42 [04:40<00:00,  6.69s/part]


DataSheetUrl  \
MouserPartNumber     Quantity                
279-CPF0805B51KE1    1                 NaN   
                     10                NaN   
                     100               NaN   
                     500               NaN   
                     1000              NaN   
...                                    ...   
963-EMK316BB7226ML-T 1000              NaN   
                     2000              NaN   
                     4000              NaN   
                     10000             NaN   
                     24000             NaN   

                                                                     Description  \
MouserPartNumber     Quantity                                                      
279-CPF0805B51KE1    1         Thin Film Resistors - SMD CPF 0805 51K 0.1% 25PPM   
                     10        Thin Film Resistors - SMD CPF 0805 51K 0.1% 25PPM   
                     100       Thin Film Resistors - SMD CPF 0805 51K 0.1% 25PPM   
                     500       Thin Film Resistors - SMD CPF 0805 51K 0.1% 25PPM   
                     1000      Thin Film Resistors - SMD CPF 0805 51K 0.1% 25PPM   
...                                                                          ...   
963-EMK316BB7226ML-T 1000      Multilayer Ceramic Capacitors MLCC - SMD/SMT 9...   
                     2000      Multilayer Ceramic Capacitors MLCC - SMD/SMT 9...   
                     4000      Multilayer Ceramic Capacitors MLCC - SMD/SMT 9...   
                     10000     Multilayer Ceramic Capacitors MLCC - SMD/SMT 9...   
                     24000     Multilayer Ceramic Capacitors MLCC - SMD/SMT 9...   

                                                                   Category  \
MouserPartNumber     Quantity                                                 
279-CPF0805B51KE1    1                            Thin Film Resistors - SMD   
                     10                           Thin Film Resistors - SMD   
                     100                          Thin Film Resistors - SMD   
                     500                          Thin Film Resistors - SMD   
                     1000                         Thin Film Resistors - SMD   
...                                                                     ...   
963-EMK316BB7226ML-T 1000      Multilayer Ceramic Capacitors MLCC - SMD/SMT   
                     2000      Multilayer Ceramic Capacitors MLCC - SMD/SMT   
                     4000      Multilayer Ceramic Capacitors MLCC - SMD/SMT   
                     10000     Multilayer Ceramic Capacitors MLCC - SMD/SMT   
                     24000     Multilayer Ceramic Capacitors MLCC - SMD/SMT   

                               LeadTime LifecycleStatus  \
MouserPartNumber     Quantity                             
279-CPF0805B51KE1    1         101 Days             NaN   
                     10        101 Days             NaN   
                     100       101 Days             NaN   
                     500       101 Days             NaN   
                     1000      101 Days             NaN   
...                                 ...             ...   
963-EMK316BB7226ML-T 1000       91 Days             NaN   
                     2000       91 Days             NaN   
                     4000       91 Days             NaN   
                     10000      91 Days             NaN   
                     24000      91 Days             NaN   

                                               Manufacturer  \
MouserPartNumber     Quantity                                 
279-CPF0805B51KE1    1         TE Connectivity / Holsworthy   
                     10        TE Connectivity / Holsworthy   
                     100       TE Connectivity / Holsworthy   
                     500       TE Connectivity / Holsworthy   
                     1000      TE Connectivity / Holsworthy   
...                                                     ...   
963-EMK316BB7226ML-T 1000             

Cant find 9 part(s):


,Qty,Value,Device,Package,Parts,Description,ARROW_PART_NUMBER,ARROW_PRICE-STOCK,DESCRIPTION,HEIGHT,MANUFACTURER,MANUFACTURER_NAME,MANUFACTURER_PART_NUMBER,MOUSER_PART_NUMBER,MOUSER_PRICE-STOCK,POPULARITY,PROD_ID,SPICEPREFIX,error_code
1,2,NaN,CONN_06SILK_FEMALE_PTH,1X06,"J2, J13",Multi connection point. Often used as Generic ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CONN-08437,NaN,30
5,2,100k,R-US_0411/15,0411/15,"R5P, R5P1","RESISTOR, American symbol",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,R,30
20,4,220,R-US_0411/15,0411/15,"R2P, R2P1, R3P, R3P1","RESISTOR, American symbol",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,R,30
23,2,300k,R-US_0411/15,0411/15,"R4P, R4P1","RESISTOR, American symbol",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,R,30
25,4,330,R-US_0411/15,0411/15,"R6P, R6P1, R6P2, R6P3","RESISTOR, American symbol",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,R,30
30,24,68u,500HXW68MEFR14.5X45,CAPPRD750W80D1475H4700,"C1P, C1P1, C1P2, C1P3, C2P, C2P1, C2P2, C2P3, ...",Aluminum Electrolytic Capacitors - Radial Lead...,NaN,NaN,Aluminum Electrolytic Capacitors - Radial Lead...,47mm,NaN,Rubycon,500HXW68MEFR14.5X45,232-500HXW68MEFR145X,https://www.mouser.com/ProductDetail/Rubycon/5...,NaN,NaN,NaN,31
38,4,TCM1A106M8R,TCM1A106M8R,CAPPM168X90N,"C5, C6, C7, C8",Tantalum Capacitors - Solid SMD 10 VDC 10 uF 0...,NaN,NaN,Tantalum Capacitors - Solid SMD 10 VDC 10 uF 0...,0.9mm,NaN,Kyocera AVX,TCM1A106M8R,755-TCM1A106M8R,https://www.mouser.co.uk/ProductDetail/KYOCERA...,NaN,NaN,NaN,31
39,2,UCY2E220MPD1TD,UCY2E220MPD1TD,CAPPRD500W60D1025H1750,"C4P, C4P1",Aluminum Electrolytic Capacitors - Radial Lead...,NaN,NaN,Aluminum Electrolytic Capacitors - Radial Lead...,17.5mm,NaN,Nichicon,UCY2E220MPD1TD,647-UCY2E220MPD1TD,https://www.mouser.co.uk/ProductDetail/Nichico...,NaN,NaN,NaN,31
40,4,VT9-PT,VT9-PT,VT9-PT,"SX1, SX2, SX3, SX4",VACUUM TUBE SOCKET,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,30


## Create Output Dataframe

In [6]:
quantities = np.unique([j for i, j in results.index.values])
quantities = quantities[quantities <= 100]

price_breaks = pd.DataFrame(np.unique([i for i, j in results.index.values]), columns = ["MouserPartNumber"])
price_breaks.set_index("MouserPartNumber", inplace = True)

for part_num in [i for i, j in results.index.values]:
    for quantity in quantities:
        result_df = results.loc[(results.index.get_level_values('MouserPartNumber') == part_num) & (results.index.get_level_values('Quantity') == quantity)]["Price"]
        if result_df.empty:
            price_breaks.at[part_num, f"Unit Price Per {quantity}"] = np.nan
        else:
            if len(result_df.values) == 1:
                price_breaks.at[part_num, f"Unit Price Per {quantity}"] = result_df.values
            elif len(set(result_df.values)) == 1:
                price_breaks.at[part_num, f"Unit Price Per {quantity}"] = result_df.values[0]
            
price_breaks = price_breaks.ffill(axis=1)

# Remove dollar signs
for idx, row in price_breaks.iterrows(): # Bear in mind that "idx" is the part name!!!
    for col_name in price_breaks.columns:
        row[col_name] = str(row[col_name]).replace('$', '')

for column in price_breaks.columns:
    try:
        price_breaks[column] = pd.to_numeric(price_breaks[column])
    except Exception as e:
        pass

# Add in columns from BOM
in_cols = ["Qty", "Value", "Device"]
out_cols = ["Per 1 Qty", "Value", "Device"]

for idx1, (in_col, out_col) in enumerate(zip(in_cols, out_cols)):
    for idx2, part_num in enumerate(price_breaks.index.values):
        qty_val = np.unique(results.loc[results.index.get_level_values('MouserPartNumber') == part_num][in_col].values)[0]
    
        if idx2 == 0:
            price_breaks.insert(loc = idx1, column = out_col, value = qty_val)
        else:
            price_breaks.at[part_num, out_col] = qty_val

# Qty
multiplier = 1
price_breaks.insert(loc = 1, column = "Qty", value = [i * multiplier for i in price_breaks["Per 1 Qty"].values])

# Extras?
extras = np.full((len(price_breaks)), True)
extras[-5] = False
price_breaks.at[:, "Extras?"] = extras

# Sub-count
for idx, row in price_breaks.iterrows():
    if row["Extras?"] and (float(row["Unit Price Per 1"]) < 1):
        price_breaks.at[idx, "Sub-count"] = np.round(row["Qty"] + 1 + row["Qty"] * 0.03)
    else:
        price_breaks.at[idx, "Sub-count"] = row["Qty"]

# Sub-totals per number
[f"Sub-total Per {quantity}" for quantity in quantities]
in_cols = [f"Unit Price Per {quantity}" for quantity in quantities]
out_cols = [f"Sub-total Per {quantity}" for quantity in quantities]

for _, (in_col, out_col) in enumerate(zip(in_cols, out_cols)):
    for idx, row in price_breaks.iterrows():
        multiplier = int(in_col.replace("Unit Price Per ", ""))
        if row["Sub-count"] > (multiplier - 1):
            price_breaks.at[idx, out_col] = row["Sub-count"] * row[in_col]
        else:
            price_breaks.at[idx, out_col] = multiplier *  row[in_col]

# Final Count
for idx, row in price_breaks.iterrows():
    # multiplier = int(in_col.replace("Unit Price Per ", ""))
    if row["Sub-count"] < int(row[out_cols].idxmin().replace("Sub-total Per ", "")):
        price_breaks.at[idx, "Final Count"] = int(row[out_cols].idxmin().replace("Sub-total Per ", ""))
    else:
        price_breaks.at[idx, "Final Count"] = row["Sub-count"]

# Final Sub-total
for idx, row in price_breaks.iterrows():
    price_breaks.at[idx, "Final Sub-total"] = price_breaks.at[idx, "Final Count"] * price_breaks.at[idx, f"Unit Price Per {int(row[out_cols].idxmin().replace("Sub-total Per ", ""))}"]

display(price_breaks)
print(f"Merchandise Total: ${np.sum(price_breaks["Final Sub-total"])}")

,Per 1 Qty,Qty,Value,Device,Unit Price Per 1,Unit Price Per 10,Unit Price Per 20,Unit Price Per 25,Unit Price Per 50,Unit Price Per 60,...,Sub-count,Sub-total Per 1,Sub-total Per 10,Sub-total Per 20,Sub-total Per 25,Sub-total Per 50,Sub-total Per 60,Sub-total Per 100,Final Count,Final Sub-total
MouserPartNumber,,,,,,,,,,,,,,,,,,,,,
279-CPF0805B51KE1,6,6,33k,CPF0805B51KE1,0.25,0.178,0.178,0.178,0.178,0.178,...,7.0,1.75,1.780,3.56,4.450,8.90,10.68,15.8,7.0,1.750
279-LR1F10K,10,10,10k,LR1F10K,0.10,0.027,0.027,0.024,0.024,0.024,...,11.0,1.10,0.297,0.54,0.600,1.20,1.44,2.2,11.0,0.297
279-LR1F1K0,8,8,1k,LR1F1K0,0.10,0.030,0.030,0.027,0.027,0.027,...,9.0,0.90,0.300,0.60,0.675,1.35,1.62,2.5,10.0,0.300
279-LR1F1M0,2,2,1M*,LR1F1M0,0.10,0.027,0.027,0.024,0.024,0.024,...,3.0,0.30,0.270,0.54,0.600,1.20,1.44,2.3,10.0,0.270
279-YR1B76R8CC,2,2,76.8,YR1B76R8CC,1.12,1.120,1.120,1.080,1.080,1.080,...,2.0,2.24,11.200,22.40,27.000,54.00,64.80,108.0,2.0,2.240
490-SJ-63033A,1,1,SJ-63033A,SJ-63033A,2.11,1.800,1.690,1.690,1.610,1.610,...,1.0,2.11,18.000,33.80,42.250,80.50,96.60,152.0,1.0,2.110
494-1N750(DO-35),2,2,1N750,1N750,2.02,2.020,2.020,2.020,2.020,2.020,...,2.0,4.04,20.200,40.40,50.500,101.00,121.20,187.0,2.0,4.040
505-DCP4G056007GD4KS,2,2,30uF/400V,DCP4G056007GD4KSSD,22.02,15.580,15.580,15.580,15.580,15.580,...,2.0,44.04,155.800,311.60,389.500,779.00,934.80,1067.0,2.0,44.040
505-MKP4G034705DKSSD,4,4,MKP4G034705D00KSSD,MKP4G034705D00KSSD,1.37,0.939,0.939,0.939,0.939,0.939,...,4.0,5.48,9.390,18.78,23.475,46.95,56.34,79.5,4.0,5.480


Merchandise Total: $188.507


## Output in Mouser Part List Import Tool Format

In [7]:
for idx, row in price_breaks.iterrows():
    print(row.name + "|" + str(int(row["Final Count"])))

279-CPF0805B51KE1|7
279-LR1F10K|11
279-LR1F1K0|10
279-LR1F1M0|10
279-YR1B76R8CC|2
490-SJ-63033A|1
494-1N750(DO-35)|2
505-DCP4G056007GD4KS|2
505-MKP4G034705DKSSD|4
505-MKS4B0312AJC|3
512-1N4148T50R|10
584-LT3042EMSE#PBF|6
594-MBB02070C2490FRP|3
595-LP5907MFX-1.8NPB|3
603-MF0207FTE52-182R|10
647-UFW1H101MPD1TD|10
651-1935200|2
651-1935255|2
652-CHP0805FX1001ELF|7
653-XR2A-0811-N|7
71-CPF2-F-15K-E3|4
71-RWR80S4020FRB12|2
754-RR1220P-204D|10
771-BZX79-C16,143|5
863-1N4007FFG|17
863-LM317MTG|3
926-LM1117MPX5.0NOPB|2
963-EMF212B7104KGHT|3
963-EMK212ABJ475KG-T|10
963-EMK212BB7106KG-T|10
963-EMK316BB7226ML-T|10
